Laurent polynomial systems are systems that have negative exponents.

A reference for the general case is the paper by Carlo Innocenti:
**Polynomial solution to the position analysis of the 7-line Assur kinematic
chain with one quaternary link**, in *Mech. Mach. Theory*, Vol. 30, No. 8,
pages 1295-1303, 1995.  

The special case was introduced in the paper with title: 
**Numerical decomposition of the solution sets of polynomial
systems into irreducible components**, *SIAM J. Numer. Anal.* 38(6):2022-2046,
2001, by Andrew Sommese, Jan Verschelde, and Charles Wampler.

This special sevenbar mechanism has 6 isolated solutions and a cubic curve.

# Design of a moving 7-bar mechanism

The notebook sets up a moving sevenbar example with sympy.
There is one irreducible component of degree three
and six isolated points.

In [1]:
from sympy import var

In [2]:
from cmath import exp

In [3]:
from phcpy.solutions import coordinates, diagnostics, condition_tables
from phcpy.solver import solve
from phcpy.sets import double_laurent_membertest
from phcpy.cascades import double_laurent_top_cascade
from phcpy.cascades import double_laurent_cascade_filter
from phcpy.factor import double_monodromy_breakup

PHCv2.4.88 released 2023-12-26 works!


## 1. A Laurent Polynomial System

The code in this section defines the Laurent polynomial system for a generic instance of the parameters.

In [4]:
def symbolic_equations():
    """
    Returns the symbolic equations,
    with parameters a1, a2, a3, a4, a5, a6
    b0, b2, b3, b4, b5, and c0, with variables
    t1, t2, t3, t4, and t5. 
    """
    a0, a1, a2, a3, a4, a5, a6 = var('a0, a1, a2, a3, a4, a5, a6')
    b0, b2, b3, b4, b5, c0 = var('b0, b2, b3, b4, b5, c0')
    t1, t2, t3, t4, t5, t6 = var('t1, t2, t3, t4, t5, t6')
    eq1 = a1*t1 + a2*t2 - a3*t3 - a0
    eq2 = b2*t2 + a3*t3 - a4*t4 + a5*t5 - b0
    eq3 = a4*t4 + b5*t5 - a6*t6 - c0
    return [eq1, eq2, eq3]

In [5]:
def generic_problem(eqs):
    """
    Given the symbolic equations in eqs,
    defines the equations for a generic problem,
    as a Laurent polynomial system.
    The system is returned as a list of string representations,
    suitable for input to the solve of phcpy.
    """
    i = complex(0, 1)
    subdict = {a0: 0.7 + 0.2*i, b0: 0.6, c0: 0.5 - 0.5*i, \
        a1: 0.7, a2: 0.8, b2: 0.6 + 0.5*i, a3: 0.4, a4: 0.6, \
        a5: 0.8, b5: 0.4 + 0.3*i, a6: 0.9}
    print(subdict)
    conjugates = {a0: 0.7 - 0.2*i, b0: 0.6, c0: 0.5 + 0.5*i, \
        a1: 0.7, a2: 0.8, b2: 0.6 - 0.5*i, a3: 0.4, a4: 0.6, \
        a5: 0.8, b5: 0.4 - 0.3*i, a6: 0.9}
    print(conjugates)
    result = []
    for equ in eqs:
        pol = equ.subs(subdict)
        result.append(str(pol) + ';')
    for equ in eqs:
        pol = str(equ.subs(conjugates))
        pol = pol.replace('t1', 't1**(-1)')
        pol = pol.replace('t2', 't2**(-1)')
        pol = pol.replace('t3', 't3**(-1)')
        pol = pol.replace('t4', 't4**(-1)')
        pol = pol.replace('t5', 't5**(-1)')
        pol = pol.replace('t6', 't6**(-1)')
        result.append(pol + ';')
    return result

In [6]:
eqs = symbolic_equations()
for equ in eqs:
    print(equ)

-a0 + a1*t1 + a2*t2 - a3*t3
a3*t3 - a4*t4 + a5*t5 - b0 + b2*t2
a4*t4 - a6*t6 + b5*t5 - c0


In [7]:
T1, T2, T3, T4, T5, T6 = var('T1, T2, T3, T4, T5, T6')
generic = generic_problem(eqs)
for equ in generic:
    print(equ)

{a0: (0.7+0.2j), b0: 0.6, c0: (0.5-0.5j), a1: 0.7, a2: 0.8, b2: (0.6+0.5j), a3: 0.4, a4: 0.6, a5: 0.8, b5: (0.4+0.3j), a6: 0.9}
{a0: (0.7-0.2j), b0: 0.6, c0: (0.5+0.5j), a1: 0.7, a2: 0.8, b2: (0.6-0.5j), a3: 0.4, a4: 0.6, a5: 0.8, b5: (0.4-0.3j), a6: 0.9}
0.7*t1 + 0.8*t2 - 0.4*t3 - 0.7 - 0.2*I;
t2*(0.6 + 0.5*I) + 0.4*t3 - 0.6*t4 + 0.8*t5 - 0.6;
0.6*t4 + t5*(0.4 + 0.3*I) - 0.9*t6 - 0.5 + 0.5*I;
0.7*t1**(-1) + 0.8*t2**(-1) - 0.4*t3**(-1) - 0.7 + 0.2*I;
t2**(-1)*(0.6 - 0.5*I) + 0.4*t3**(-1) - 0.6*t4**(-1) + 0.8*t5**(-1) - 0.6;
0.6*t4**(-1) + t5**(-1)*(0.4 - 0.3*I) - 0.9*t6**(-1) - 0.5 - 0.5*I;


In [8]:
sols = solve(generic)
print('found', len(sols), 'solutions')

found 18 solutions


In [9]:
for (idx, sol) in enumerate(sols):
    print('Solution', idx+1, ':')
    print(sol)

Solution 1 :
t :  1.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 t1 :  9.57692499414695E-01  -2.87793461643650E-01
 t2 :  1.52466396219873E-01   9.88308655240722E-01
 t3 :  2.30894666415461E-01   9.72978752605057E-01
 t4 : -7.15463283586449E-01   6.98650334459017E-01
 t5 :  6.01298316462979E-01  -7.99024614526228E-01
 t6 : -4.98945843565344E-01   8.66633166448681E-01
== err :  1.612E-15 = rco :  9.853E-02 = res :  9.437E-16 =
Solution 2 :
t :  1.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 t1 : -3.72672721084468E-01   8.34512378383221E-01
 t2 :  6.76643775704828E-01  -6.09422676945313E-01
 t3 : -1.04888971048816E+00  -2.58448691719989E-01
 t4 :  9.65564818521611E-01  -1.06052984170056E-01
 t5 :  1.11024646426585E+00   8.38492556259197E-02
 t6 :  5.53647444590589E-01   8.92202056697877E-01
== err :  1.184E-15 = rco :  2.023E-02 = res :  9.576E-16 =
Solution 3 :
t :  1.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 

Observe from the diagnostics that all 18 solutions are well conditioned.

In [10]:
condition_tables(sols)

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 2],
 [3, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 7])

## 2. A Special Problem

As we have as many equations as variables, for general coefficients, we will have only isolated solutions.  For special parameters, the system has an irreducible cubic as a solution set.

In [11]:
def special_parameters():
    """
    Returns a dictionary with special values for the parameters
    for the Assur7c in Roberts Cognate pattern.
    Before calling this function, the symbolic_equations()
    must have defined the variables for the parameters.
    """
    i = complex(0, 1)
    # start with the independent parameters
    result = {b0: 0.0, c0: 1.2, a2: 0.46, \
        b2: -0.11 + 0.49*i, a5: 0.41}
    theta4 = 0.6 + 0.8*i
    theta3 = exp(1.8*i)
    # add the derived parameters
    result[a3] = result[a5]
    beta = result[b2]/result[a2]
    result[a0] = result[c0]/beta
    result[b5] = result[a5]*beta
    result[a4] = abs(result[b2])
    result[a1] = abs(result[a0] + result[a3]*theta3 - result[a4]*theta4/beta)
    result[a6] = abs(result[a4]*theta4 - result[b5]*theta3-result[c0])
    return result

In [12]:
def conjugates(dic):
    """
    Given on input a dictionary with variables as keys
    and complex numbers as values.
    Returns a dictionary with the same keys,
    but with values replaced by complex conjugates.
    """
    result = {}
    for key in list(dic.keys()):
        result[key] = dic[key].conjugate()
    return result

In [13]:
def special_problem(eqs):
    """
    Given the symbolic equations in eqs,
    replaces the parameters with special values.
    """
    pars = special_parameters()
    conj = conjugates(pars)
    result = []
    for equ in eqs:
        pol = equ.subs(pars)
        result.append(str(pol) + ';')
    for equ in eqs:
        pol = str(equ.subs(conj))
        pol = pol.replace('t1', 't1**(-1)')
        pol = pol.replace('t2', 't2**(-1)')
        pol = pol.replace('t3', 't3**(-1)')
        pol = pol.replace('t4', 't4**(-1)')
        pol = pol.replace('t5', 't5**(-1)')
        pol = pol.replace('t6', 't6**(-1)')
        result.append(pol + ';')
    return result

In [14]:
special = special_problem(eqs)
for equ in special:
    print(equ)

0.710358341606049*t1 + 0.46*t2 - 0.41*t3 + 0.240761300555115 + 1.07248215701824*I;
t2*(-0.11 + 0.49*I) + 0.41*t3 - 0.502195181179589*t4 + 0.41*t5;
0.502195181179589*t4 + t5*(-0.0980434782608696 + 0.436739130434783*I) - 0.775518556663656*t6 - 1.2;
0.710358341606049*t1**(-1) + 0.46*t2**(-1) - 0.41*t3**(-1) + 0.240761300555115 - 1.07248215701824*I;
t2**(-1)*(-0.11 - 0.49*I) + 0.41*t3**(-1) - 0.502195181179589*t4**(-1) + 0.41*t5**(-1);
0.502195181179589*t4**(-1) + t5**(-1)*(-0.0980434782608696 - 0.436739130434783*I) - 0.775518556663656*t6**(-1) - 1.2;


In [15]:
sols = solve(special)
print('found', len(sols), 'solutions')

found 6 solutions


In [16]:
for (idx, sol) in enumerate(sols):
    print('Solution', idx+1, ':')
    print(sol)

Solution 1 :
t :  1.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 t1 : -5.66058532229597E-01  -3.67759063106022E-01
 t2 :  5.87958784214731E-01  -2.31024744989669E-01
 t3 :  2.66141319725173E-01   1.71943916132782E+00
 t4 :  2.42136902539446E-01   1.56435563278512E+00
 t5 : -8.79136932379914E-02  -5.67977370543055E-01
 t6 : -1.05957839449340E+00   1.03531112257767E+00
== err :  3.073E-15 = rco :  3.135E-02 = res :  9.853E-16 =
Solution 2 :
t :  1.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 t1 :  3.53717078322846E-01  -5.30879118400565E-01
 t2 : -5.99365365166588E-01  -1.57888836421937E+00
 t3 :  5.27607584716191E-01  -7.54168308853111E-02
 t4 :  5.86169848996842E-01  -8.37877878416834E-02
 t5 : -1.85739737768408E+00   2.65498503011424E-01
 t6 : -1.08247082572556E+00  -1.13383016827930E+00
== err :  2.542E-15 = rco :  3.053E-02 = res :  4.510E-16 =
Solution 3 :
t :  1.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 

In [17]:
condition_tables(sols)

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 1],
 [2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6])

The `solve` of the solver module misses the component, but finds all isolated solutions.

## 3. A Numerical Irreducible Decomposition

A numerical irreducible decomposition for this system augments the system with a linear equation and adds one slack variable.  A cascade of homotopies find generic points on all positive dimensional components of the solution set.

In [18]:
def embed_and_cascade(pols, topdim):
    """
    Computes and solves an embedding at top dimension topdim
    of the Laurent polynomials in pols, before running one
    step in the cascade homotopy.
    Returns the embedded system, the three generic points,
    and the filtered solutions at the end of the cascade.
    """
    (embpols, sols0, sols1) \
        = double_laurent_top_cascade(len(pols), topdim, pols, 1.0e-08)
    print('the top generic points :')
    for (idx, sol) in enumerate(sols0):
        print('Solution', idx+1, ':')
        print(sol)
    print('the nonsolutions :')
    for (idx, sol) in enumerate(sols1):
        print('Solution', idx+1, ':')
        print(sol)
    print('... running cascade step ...')
    (embdown, nsols1, sols2) = double_laurent_cascade_filter(1, embpols, \
        sols1, 1.0e-8)
    filtsols2 = []
    for (idx, sol) in enumerate(nsols1):
        err, rco, res = diagnostics(sol)
        if res < 1.0e-8 and rco > 1.0e-8:
            _, point = coordinates(sol)
            crdpt = [] 
            for pt in point:
                crdpt.append(pt.real)
                crdpt.append(pt.imag)
            onset = double_laurent_membertest(embpols, sols0, 1, crdpt) 
            if not onset:
                filtsols2.append(sol)
    print('... after running the cascade ...')
    for (idx, sol) in enumerate(filtsols2):
        print('Solution', idx+1, ':')
        print(sol)
    print('found %d isolated solutions' % len(filtsols2))
    return (embpols, sols0, filtsols2)

In [19]:
(embpols, sols0, isosols) = embed_and_cascade(special, 1)

the top generic points :
Solution 1 :
t :  1.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 t1 : -1.38546903964616E-01  -6.90082116709782E-01
 t2 :  1.73335780614296E+00  -1.87923014523194E+00
 t3 :  2.29192181084370E+00  -6.88217799479036E-01
 zz1 :  6.81370071005700E-17   1.65228741665612E-16
 t4 :  1.45392357364500E+00   2.10288883797136E+00
 t5 : -2.29192181084370E+00   6.88217799479037E-01
 t6 : -7.03671420729004E-01  -1.59719770241003E-02
== err :  1.018E-14 = rco :  7.104E-03 = res :  1.619E-15 =
Solution 2 :
t :  1.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 t1 :  9.95322784254993E-01   2.68098302301308E-02
 t2 : -2.29577075969873E-01  -2.62541647600257E-01
 t3 :  2.05412606828065E+00   2.36770142844668E+00
 zz1 :  7.58249556746807E-16  -7.20127010307089E-16
 t4 :  3.06452334567059E-01  -1.66495396855090E-01
 t5 : -2.05412606828065E+00  -2.36770142844668E+00
 t6 :  2.44172639794733E-01  -9.65280235905487E-01
== err :  3.577E-15 =

In [20]:
for pol in embpols:
    print(pol)

 + 7.10358341606049E-01*t1 + 4.60000000000000E-01*t2 - 4.10000000000000E-01*t3 + (-3.99034499614690E-01 + 9.16935912764493E-01*i)*zz1+(2.40761300555115E-01 + 1.07248215701824E+00*i);
 + (-1.10000000000000E-01 + 4.90000000000000E-01*i)*t2 + 4.10000000000000E-01*t3 - 5.02195181179589E-01*t4 + 4.10000000000000E-01*t5 + (-2.71603706128330E-01-9.62409178477302E-01*i)*zz1;
 + 5.02195181179589E-01*t4 + (-9.80434782608696E-02 + 4.36739130434783E-01*i)*t5 - 7.75518556663656E-01*t6 + (-9.98029449494905E-01 + 6.27472544490738E-02*i)*zz1 - 1.20000000000000E+00;
 + (-9.58594885352021E-01-2.84773323499490E-01*i)*zz1+(2.40761300555115E-01-1.07248215701824E+00*i) - 4.10000000000000E-01*t3^-1 + 4.60000000000000E-01*t2^-1 + 7.10358341606049E-01*t1^-1;
 + (9.19472457329587E-01-3.93154422857342E-01*i)*zz1 + 4.10000000000000E-01*t5^-1 - 5.02195181179589E-01*t4^-1 + 4.10000000000000E-01*t3^-1 + (-1.10000000000000E-01-4.90000000000000E-01*i)*t2^-1;
 + (4.61876615285503E-01 + 8.86944187788841E-01*i)*zz1 - 1.2

In [21]:
print('the isolated solutions :')
for (idx, sol) in enumerate(isosols):
    print('Solution', idx+1, ':')
    print(sol)

the isolated solutions :
Solution 1 :
t :  1.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 t1 : -5.45421549881402E-01  -8.38161877518281E-01
 t2 : -5.49340801920855E-01  -8.35598398361888E-01
 t3 : -9.74098087752273E-01   2.26125884049935E-01
 t6 : -8.74935009483375E-01  -4.84240363022669E-01
 t4 :  1.78912046655936E-01  -9.83865071827120E-01
 t5 :  4.72153176961023E-02  -9.98884734979395E-01
== err :  3.858E-16 = rco :  3.659E-02 = res :  1.388E-16 =
Solution 2 :
t :  1.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 t1 : -5.66058532229596E-01  -3.67759063106022E-01
 t2 :  5.87958784214732E-01  -2.31024744989669E-01
 t3 :  2.66141319725174E-01   1.71943916132782E+00
 t6 : -1.05957839449340E+00   1.03531112257767E+00
 t4 :  2.42136902539447E-01   1.56435563278512E+00
 t5 : -8.79136932379912E-02  -5.67977370543055E-01
== err :  1.140E-15 = rco :  2.128E-02 = res :  4.441E-16 =
Solution 3 :
t :  1.00000000000000E+00   0.00000000000000E+00
m :

Given the witness set, monodromy is applied to factor.

In [22]:
fac = double_monodromy_breakup(embpols, sols0, 1, islaurent=True, verbose=True)

... running monodromy loops in double precision ...
... initializing the grid for the linear trace ...
The diagnostics of the trace grid :
  largest error on the samples : 1.1914895647759973e-14
  smallest distance between the samples : 2.3530539196335214
... starting loop 1 ...
new permutation : [3, 2, 1]
number of factors : 3 -> 2
the decomposition :
  factor 1 : ([1, 3], 0.33179448122518296)
  factor 2 : ([2], 0.3317944812254451)
... starting loop 2 ...
new permutation : [2, 3, 1]
number of factors : 2 -> 1
the decomposition :
  factor 1 : ([1, 2, 3], 4.11226608321158e-13)


In [23]:
fac

[([1, 2, 3], 4.11226608321158e-13)]

The grouping of the generic sets in one set shows that the solution curve is an irreducible cubic.